<a href="https://colab.research.google.com/github/kimhyeongmin-khu/MLMovieProj/blob/main/naver_api_to_trendData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#네이버 웹 api를 통해서 트렌드 데이터 수집
import os
import sys
import urllib.request
import json
import pandas as pd
from datetime import datetime, timedelta

client_id =
client_secret =
url = "https://openapi.naver.com/v1/datalab/search"

df = pd.read_csv('output_to_trend.csv')
final_df_list = []

for index, row in df.iterrows():
    movie_title = row['Movie_Title']
    release_date = datetime.strptime(row['Release_Date'], '%Y-%m-%d')

    # 개봉일 전후 1주일 기간 설정
    end_date = (release_date + timedelta(days=6)).strftime('%Y-%m-%d')
    start_date = release_date.strftime('%Y-%m-%d')

    # JSON 형식의 본문 문자열을 수정
    body = f"""
    {{
        "startDate": "{start_date}",
        "endDate": "{end_date}",
        "timeUnit": "date",
        "keywordGroups": [
            {{
                "groupName": "{movie_title}",
                "keywords": ["{movie_title}"]
            }}
        ]
    }}
    """

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    request.add_header("Content-Type", "application/json")

    try:
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read()
            response_data = json.loads(response_body.decode('utf-8'))

            # 결과 리스트에서 데이터 추출
            results = response_data['results'][0]
            title = results['title']
            data = results['data']

            # 데이터프레임 생성
            df_temp = pd.DataFrame(data)

            # period를 날짜 순서로 정렬하고, 인덱스 재설정
            df_temp = df_temp.sort_values(by='period').reset_index(drop=True)

            # 인덱스를 1부터 7로 설정
            df_temp.index = range(1, len(df_temp) + 1)

            # 컬럼 이름 설정
            df_temp.columns = ['column', title]

            # 컬럼을 1부터 7로 설정
            df_temp['column'] = range(1, len(df_temp) + 1)

            # 최종 데이터프레임에 추가
            final_df_list.append(df_temp.set_index('column').T)

        else:
            print("Error Code:" + str(rescode))
    except Exception as e:
        print(f"Error: {e}")

# 모든 데이터프레임을 수직으로 결합
final_df = pd.concat(final_df_list)

# 결과를 CSV 파일로 저장
final_df.to_csv('combined_trend_data.csv', index=True)

# 결과 출력
print(final_df)
